## **Генератор упражнений по английскому языку - Read books in english ("Red Hat")**

**Заказчик:** Школа английского языка Яндекс

**Цель:** Создание приложения, которое автоматически преобразует предоставленный текст в увлекательные задания по английскому языку, используя технологии обработки естественного языка (NLP).

**Задача:**
- Создать приложение, которое преобразует текст в набор упражнений по английскому языку (поиск и подбор синонимов, антонимов, преобразование слов - времена глагола, степени прилагательных, единственное/множественное число, поиск части речи и определение структуры предложенияи т.п.).
- Разработать модуль (класс или набор функций), осуществляющий преобразование текста.
- Подготовить датасет, включающий каждое предложение, тип упражнения, преобразованное предложение с пропущенными словами, варианты ответов и правильный ответ.
- Дополнительная задача: создать обертку для модуля через платформу Streamlit.

За основу была взята сказка Шарля Перро "Красная шапочка": Little_Red_Riding_Hood_Charles_Perrault

## **Загрузка библиотек**

In [ ]:
#Установка библиотеки transformers и др.для  GC
import sys
ENV_COLAB = 'google.colab' in sys.modules

if ENV_COLAB:
    !pip install pytorch-transformers
    !pip install transformers
    !pip install pytorch-pretrained-bert
    import transformers as ppb

    print('Environment: Google Colab')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 53.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.6 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=94cc12cbc3ada4fec74366926a5b3993e692d2eaac903dc54d1102307ec18ccf
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
!pip install pyinflect
#https://pypi.org/project/pyinflect/
import pyinflect
!pip install sentence_splitter
from sentence_splitter import SentenceSplitter, split_text_into_sentences
#Fixes contractions such as `you're` to you `are`
!pip install contractions
import contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.5/703.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 6.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from numpy.random import RandomState
from sklearn.utils import shuffle
import random
import time

from tqdm import notebook, tqdm, trange
import re
import nltk
nltk.download('wordnet','stopwords','punkt','averaged_perceptron_tagger')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

import json
import spacy
import en_core_web_sm
import gensim.downloader as api
from gensim.models import Word2Vec

from joblib import dump
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# малая модель spacy
nlp = en_core_web_sm.load()
#nlp = spacy.load("en_core_web_sm")
# малая модель glove wiki
# внимание - очень долго скачивает, если она еще не установлена
model = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


## Загрузка и предобработка текста

In [ ]:
with open('/content/Little_Red_Riding_Hood_Charles_Perrault.txt', 'r') as f: #, encoding = 'utf-8'
    text = f.read()
# убираем сокращенные формы глаголов
text = contractions.fix(text)
#print(text)

In [ ]:
#делим текст на предложения - формируем таблицу
splitter = SentenceSplitter(language='en')
data = splitter.split(text=text)
data = pd.DataFrame({'row': data})

In [ ]:
#убираем лишние пустые строки
data = data.replace('', np.nan)
data = data.dropna().reset_index(drop=True)
data.head(10)

,row
0,Little Red Riding Hood.
1,Charles Perrault.
2,Once upon a time there lived in a certain vill...
3,Her mother was excessively fond of her; and he...
4,This good woman had a little red riding hood m...
5,It suited the girl so extremely well that ever...
6,"One day her mother, having made some cakes, sa..."
7,"Take her a cake, and this little pot of butter."""
8,Little Red Riding Hood set out immediately to ...
9,"As she was going through the wood, she met wit..."


In [ ]:
# Создание нового столбца с 4 случайными значениями
types = ['select_word', 'select_sentence', 'missing_word', 'noun_phrases']
description = ['Выберете слово', 'Какое предложение верно?', 'Заполните пропуск', 'Чем является выделенная фраза/слово?']
random_data = np.random.choice(types, len(data))
#data = data.assign(type=random_data)
random_data = np.random.choice(types, len(data))
data['type'] = random_data
data['description'] = [description[types.index(t)] for t in random_data]

data.head(10)
#data = pd.DataFrame(columns=['raw', 'type', 'object', 'options', 'answer', 'description'])

,row,type,description
0,Little Red Riding Hood.,select_word,Выберете слово
1,Charles Perrault.,select_sentence,Какое предложение верно?
2,Once upon a time there lived in a certain vill...,missing_word,Заполните пропуск
3,Her mother was excessively fond of her; and he...,select_sentence,Какое предложение верно?
4,This good woman had a little red riding hood m...,noun_phrases,Чем является выделенная фраза/слово?
5,It suited the girl so extremely well that ever...,noun_phrases,Чем является выделенная фраза/слово?
6,"One day her mother, having made some cakes, sa...",noun_phrases,Чем является выделенная фраза/слово?
7,"Take her a cake, and this little pot of butter.""",missing_word,Заполните пропуск
8,Little Red Riding Hood set out immediately to ...,select_sentence,Какое предложение верно?
9,"As she was going through the wood, she met wit...",missing_word,Заполните пропуск


In [ ]:
data1=data.copy()
#количество слов в предложении
def count_words(data):
    doc = nlp(data)
    return len(doc)
data1['number_words'] = data1['row'].apply(lambda x: len(nlp(x)))
#data1.head()

## Создаем функции для формирования упражнений

In [ ]:
#'select_word'/'missing_word' - глагол
# изменение формы глагола с помощью pyinflect
def verb_answer(data, index):
    verb1 = []
    count = 0
    for token in nlp(data):
    #for token in nlp(str(data)):
        if token.pos_ == 'VERB':
            count += 1
            if count == index:
                verb1.append(token.text)
                #verb = token.lemma_
    return verb1

def form_verb(data, index):
    verb_last = []
    count = 0
    for token in nlp(data):
        if token.pos_=='VERB':
            count += 1
            if count == index:
                #verb = token.text
                verb_last.append(token._.inflect('VBP')) #"Базовая форма:"
                verb_last.append(token._.inflect('VBZ')) #"Present tense, 3rd person:"
                verb_last.append(token._.inflect('VBG')) #"Present continous tense:"
                verb_last.append(token._.inflect('VBD')) # "Past tense:"
                break
    return verb_last

In [ ]:
#'select_sentence'/ 'выберете предложение'
# заменим существительные, глаголы, причастия и прилагательные
# на случайные близкие слова и анти-слова
#sentence =[sent]
def select_sent (sent):
    sent = sent #'Where are you going so early, Little Red Cap?'
    new_sent_1, new_sent_2 = sent, sent
    i=5
    sentence =[sent]
    for token in nlp(sent):
        if token.pos_ in ['VERB',  'ADV'] :  #['NOUN',  'ADJ']
            try:
                m, n = np.random.randint(0, i, 2)

                new_word_1 = model.most_similar(token.text.lower(), topn=i)[m][0]
                new_word_2 = model.most_similar(positive = [token.text.lower(), 'bad'],
                                            negative = ['good'],
                                            topn=i)[n][0]
                new_word_1 = new_word_1.title() if token.text.istitle() else new_word_1
                new_word_2 = new_word_2.title() if token.text.istitle() else new_word_2

                new_sent_1 = new_sent_1.replace(token.text, new_word_1)
                new_sent_2 = new_sent_2.replace(token.text, new_word_2)
            except:
                pass
    sentence.append(new_sent_1)
    sentence.append(new_sent_2)

    return sentence #sent, new_sent_1, new_sent_2

In [ ]:
#noun_phrases/Чем является выделенная фраза
def extract_longest_noun_phrase(text):
    doc = nlp(text)
    longest_phrase = max(doc.noun_chunks, key=lambda chunk: len(chunk.text))
    return longest_phrase.text

def noun_phrase_longest(text):
    #noun_phrase = []
    doc = nlp(text)
    #longest_phrase1 = max([spacy.explain(chunk.root.dep_) for chunk in doc.noun_chunks], key=len)
    noun_phrase = max([spacy.explain(chunk.root.dep_) for chunk in doc.noun_chunks], key=len)
    #noun_phrase.append(longest_phrase)
    return noun_phrase
def dictionary(text):
    dictionary = [spacy.explain(chunk.root.dep_) for chunk in nlp(text).noun_chunks]
    return dictionary

### Формируем итоговую таблицу

In [ ]:
def collect_table(data, index):
    data['object'] = data.apply(lambda row: verb_answer(row['row'], index)
                                            if row['type'] == 'missing_word' else None, axis=1)\
                    .combine_first(data.apply(lambda row: verb_answer(row['row'], index)
                                            if row['type'] == 'select_word' else None, axis=1))\
                    .combine_first(data.apply(lambda row: row['row']
                                            if row['type'] == 'select_sentence' else None, axis=1))\
                    .combine_first(data.apply(lambda row: extract_longest_noun_phrase(row['row'])
                                            if row['type'] == 'noun_phrases' and list(nlp(row['row']).noun_chunks) else None, axis=1))
    data['options'] = data.apply(lambda row: [] if row['type'] == 'missing_word' else None, axis=1)\
                    .combine_first(data.apply(lambda row: form_verb(row['row'], index)
                                            if row['type'] == 'select_word' else None, axis=1))\
                    .combine_first(data.apply(lambda row: select_sent(row['row'])
                                            if row['type'] == 'select_sentence' else None, axis=1))\
                    .combine_first(data.apply(lambda row: dictionary(row['row'])
                                            if row['type'] == 'noun_phrases' else None, axis=1))
    data['answer'] = data.apply(lambda row: verb_answer(row['row'], index)
                                            if row['type'] == 'missing_word' else None, axis=1)\
                    .combine_first(data.apply(lambda row: verb_answer(row['row'], index)
                                            if row['type'] == 'select_word' else None, axis=1))\
                    .combine_first(data.apply(lambda row: row['row']
                                            if row['type'] == 'select_sentence' else None, axis=1))\
                    .combine_first(data.apply(lambda row: noun_phrase_longest(row['row'])
                                            if row['type'] == 'noun_phrases' and list(nlp(row['row']).noun_chunks) else None, axis=1))
    return data

In [ ]:
data1 = collect_table(data1, 1)
data1.head()

,row,type,description,number_words,object,options,answer
0,Little Red Riding Hood.,select_word,Выберете слово,5,[],[],[]
1,Charles Perrault.,select_sentence,Какое предложение верно?,3,Charles Perrault.,"[Charles Perrault., Charles Perrault., Charles...",Charles Perrault.
2,Once upon a time there lived in a certain vill...,missing_word,Заполните пропуск,23,[lived],[],[lived]
3,Her mother was excessively fond of her; and he...,select_sentence,Какое предложение верно?,17,Her mother was excessively fond of her; and he...,[Her mother was excessively fond of her; and h...,Her mother was excessively fond of her; and he...
4,This good woman had a little red riding hood m...,noun_phrases,Чем является выделенная фраза/слово?,13,a little red riding hood,"[nominal subject, direct object, object of pre...",object of preposition


In [ ]:
#фунцкия для упорядочивания столбцов таблицы
def swap_columns(data, column1, column2, column3, column4, column5, column6):
    data[column1], data[column2], data[column3], data[column4], data[column5], data[column6] = \
    data[column4], data[column3], data[column5], data[column6], data[column1], data[column2]
    data.rename(columns={column1: column4, column2: column3, column3: column5, \
                         column4: column6,column5: column1, column6: column2}, inplace=True)
    return data
#data_swap = swap_columns(data1, 'row', 'type', 'description', 'answer', 'object', 'options')
#data_swap.head()

In [ ]:
#выводим итоговый файл
data1.to_csv('data_red_cap.csv', index=False)

## **Вывод:**
- Подготовлен и обработан текст сказки.
- Сформированы 4 типа заданий, сформирована таблица, подготовлена к "заворачиванию" в обложку.
- Распределение упражнений идет по предложениям случайным образом.

ПРОЕКТ в процессе доработки

Планируется:
- Доработка кода, комбинирования заданий.
- Формирование приложения с помощью streamlit.
